In [ ]:
import textwrap
import json
import openai
import torch
from llmcam.fn_to_fc import complete, tool_schema, form_msg, capture_youtube_live_frame_and_save, ask_gpt4v_about_image_file

In [ ]:
def detect_objects(
        image_path: str, # Path/URL of image
        conf: float=0.05 # Confidence threshold
    ) -> str: # JSON format of detection results
    """Run YOLO object detection on an input image."""
    model = torch.hub.load("ultralytics/yolov5", "yolov5s")
    model.conf = conf  # NMS confidence threshold
    
    results = model(image_path)
    count = results.pandas().xyxy[0]['name'].value_counts().to_dict()
    print(count)
    return json.dumps(count, indent=2)

In [ ]:
YTLiveTools = [tool_schema(fn) for fn in (capture_youtube_live_frame_and_save, ask_gpt4v_about_image_file, detect_objects)]

In [ ]:
def exec_func(name, tools=[], **kwargs):
    res = globals().get(name)(**kwargs)
    return res

In [ ]:
#| eval: false

messages = [
    form_msg("system", "You are a helpful system administrator. Use the supplied tools to assist the user."),
    form_msg("user", "Hi, can you capture YouTube Live?")
]
messages.append(form_msg("user", "You can use the default link."))
messages.append(form_msg("user", "Can you extract information from this file?"))
messages.append(form_msg("user", "Use your new tool to detect."))
complete(messages, YTLiveTools, exec_func)
for message in messages:
    print(f">> {message['role'].capitalize()}:")
    try:
        print(textwrap.fill(message["content"], 100))
    except:
        print(message)

Using cache found in /home/abc/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-11-10 Python-3.10.15 torch-2.5.1+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/home/abc/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


{'car': 41, 'person': 13, 'boat': 11, 'truck': 7}
>> System:
You are a helpful system administrator. Use the supplied tools to assist the user.
>> User:
Hi, can you capture YouTube Live?
>> Assistant:
Yes, I can help capture a frame from a YouTube Live stream. Please provide me with the link to the
YouTube Live stream you want to capture from.
>> User:
You can use the default link.
>> Assistant:
{'content': None, 'refusal': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_5PBpbRqpDFBBTk3hqEaoNHYC', 'function': {'arguments': '{"link":null,"place":null}', 'name': 'capture_youtube_live_frame_and_save'}, 'type': 'function'}]}
>> Tool:
{"link": null, "place": null, "capture_youtube_live_frame_and_save":
"../data/cap_2024.11.13_10:20:56_unclear.jpg"}
>> Assistant:
I have captured a frame from the YouTube Live stream. If you need any analysis or information about
the captured image, let me know!
>> User:
Can you extract information from this file?
>> Assistant:
{'content': None, 'refusa